In [2]:
import sympy as sp
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
import model as mdl
%matplotlib notebook
plt.ion()

def angles_to_pwm(angles):
    return 300 + 150 * angles[-4:-1]/np.pi
    

### Goal
In the body frame, we have the $i$ th leg 
$$ (x,y,z)_i^T = f_i(\theta^i_t, \theta^i_m, \theta^i_b)$$
we want to choose a function $\theta^i_t, \theta^i_m, \theta^i_b = \gamma(s)$ such that $s \in [0, 1]$

Each leg consists of two 4R linkages and a parallelogram.

Two key pivot points are given by the horizontal plane of the D brace.

The two RRRR linkages define two derived angles.

In [3]:
mdl.generate_leg_position_code()

(-37.5*c_0 + 37.5*c_1 + 75.0*c_r - 7.5*s_0 - 7.5*s_1 - 16.0*s_2 - 16.0*s_3 - 66.0*s_a - 32.0*s_h - 15.0*s_r + 42.0,
 15.0*c_1 + 32.0*c_3 + 66.0*c_a + 75.0*s_1 + 42.0,
 -75.0*c_r + 32.0*s_h + 15.0*s_r,
 ({s_0: sin(bodyangle - radial_angle),
   s_1: sin(bodyangle + radial_angle),
   s_2: sin(bodyangle - height_angle),
   s_3: sin(bodyangle + height_angle),
   s_4: -sin(height_angle - radial_angle),
   s_5: sin(height_angle + radial_angle)},
  {c_0: cos(bodyangle - radial_angle),
   c_1: cos(bodyangle + radial_angle),
   c_2: cos(bodyangle - height_angle),
   c_3: cos(bodyangle + height_angle),
   c_4: cos(height_angle - radial_angle),
   c_5: cos(height_angle + radial_angle)}))

In [ ]:
(x1,x2,x3,_,_,_), p,_ = mdl.get_leg_position()

In [ ]:
X = sp.Matrix([[x1, x2, x3], [x1, x2, x3], [x1, x2, x3]])
X_p = X + X.T
X_m = X - X.T


In [ ]:
x2

In [ ]:
X_m

In [ ]:
x, p, c = mdl.get_leg_position()